# RideWise Customer Analytics Project  
## Notebook 01: Data Audit & Table Relationships

### Project Context
RideWise is a fictional European ride-hailing company operating across multiple cities.  
The company faces a **high customer churn rate** and lacks a unified, data-driven system to:
- Understand customer behavior
- Segment customers meaningfully
- Predict churn before it happens
- Design effective promotion strategies

This project aims to build the **analytical foundation** for a customer segmentation and churn prediction system using structured behavioral data.

---

### Purpose of This Notebook
The goal of this notebook is to perform a **data audit and relationship analysis** before any modeling begins.

Specifically, this notebook will:
1. Load and inspect all five datasets
2. Understand the structure and contents of each table
3. Validate primary and foreign key relationships
4. Identify missing values and potential data quality issues
5. Establish which tables should be joined and **how** they should be joined

This step is critical to avoid:
- Incorrect joins
- Data leakage
- Duplicated records
- Silent data quality bugs

---

### Datasets Used
This project uses the following datasets:

- **riders.csv**  
  Customer-level information (demographics, loyalty status, signup details)

- **trips.csv**  
  Trip-level transactional data (usage frequency, spend, recency)

- **sessions.csv**  
  App engagement data (session frequency, duration, conversion behavior)

- **promotions.csv**  
  Marketing campaign metadata and targeting rules

- **drivers.csv**  
  Driver and supply-side operational data (used for context only)

All datasets are located in the relative folder:  
`../../data/`

---

> **The unit of analysis is the customer (rider).**

This means:
- Final modeling datasets will contain **one row per rider**
- Transactional and session-level data will be **aggregated before joining**
- Joins are performed only when they serve a clear analytical purpose

---

### What Comes Next
After completing this notebook, the next steps will be:
- Defining customer churn using behavioral inactivity
- Feature engineering from trips and sessions
- Customer segmentation using clustering
- Churn prediction modeling
- Business interpretation and strategy recommendations

This notebook lays the groundwork for all downstream analysis.


### Imports libraries & display settings

In [1]:
import pandas as pd
import numpy as np

# Make pandas output easier to read
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 120)

print("Libraries imported. Ready!")

Libraries imported. Ready!


### Set the data folder path

In [6]:
DATA_DIR = '../data/'
print("Data directory set to:", DATA_DIR)

Data directory set to: ../data/


### Load the datasets

In [7]:
riders = pd.read_csv(DATA_DIR + "riders.csv")
trips = pd.read_csv(DATA_DIR + "trips.csv")
sessions = pd.read_csv(DATA_DIR + "sessions.csv")
promotions = pd.read_csv(DATA_DIR + "promotions.csv")
drivers = pd.read_csv(DATA_DIR + "drivers.csv")

print("Loaded:")
print("riders:", riders.shape)
print("trips:", trips.shape)
print("sessions:", sessions.shape)
print("promotions:", promotions.shape)
print("drivers:", drivers.shape)

Loaded:
riders: (10000, 8)
trips: (200000, 16)
sessions: (50000, 8)
promotions: (20, 11)
drivers: (5000, 7)


### Peek at each dataset

In [8]:
datasets = {
    "riders": riders,
    "trips": trips,
    "sessions": sessions,
    "promotions": promotions,
    "drivers": drivers
}

for name, df in datasets.items():
    print("\n" + "="*60)
    print(f"{name.upper()}  |  shape = {df.shape}")
    display(df.head(3))


RIDERS  |  shape = (10000, 8)


,user_id,signup_date,loyalty_status,age,city,avg_rating_given,churn_prob,referred_by
0,R00000,2025-01-24,Bronze,34.729629,Nairobi,5.0,0.142431,R00001
1,R00001,2024-09-09,Bronze,34.571020,Nairobi,4.7,0.674161,NaN
2,R00002,2024-09-07,Bronze,47.133960,Lagos,4.2,0.510379,NaN



TRIPS  |  shape = (200000, 16)


,trip_id,user_id,driver_id,fare,surge_multiplier,tip,payment_type,pickup_time,dropoff_time,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,weather,city,loyalty_status
0,T000000,R05207,D00315,12.11,1.0,0.00,Card,2024-11-27 18:41:50+02:27,2024-11-27 19:33:50+02:27,-1.108123,36.912209,-1.068155,36.875377,Foggy,Nairobi,Bronze
1,T000001,R09453,D03717,8.73,1.0,0.02,Card,2024-10-28 23:13:48+00:14,2024-10-28 23:26:48+00:14,6.675266,3.515740,6.641734,3.525620,Sunny,Lagos,Gold
2,T000002,R00567,D02035,19.68,1.0,0.00,Card,2025-02-17 05:36:41+02:27,2025-02-17 05:52:41+02:27,-1.248589,37.010668,-1.273182,37.018586,Cloudy,Nairobi,Bronze



SESSIONS  |  shape = (50000, 8)


,session_id,rider_id,session_time,time_on_app,pages_visited,converted,city,loyalty_status
0,S000000,R08605,2025-04-27 18:57:06+02:05,79,4,1,Cairo,Bronze
1,S000001,R08823,2025-04-27 07:32:22+02:27,101,3,0,Nairobi,Silver
2,S000002,R05342,2025-04-27 23:17:25+02:05,12,1,0,Cairo,Bronze



PROMOTIONS  |  shape = (20, 11)


,promo_id,promo_name,promo_type,promo_value,start_date,end_date,target_segment,city_scope,ab_test_groups,test_allocation,success_metric
0,P000,Peak Hour Pass,surge_waiver,1.0,2025-04-26,2025-05-25,All,Nairobi,['All'],[1.0],Usage Frequency
1,P001,Peak Hour Pass,surge_waiver,1.0,2025-04-26,2025-05-22,All,Cairo,"['Control', 'Variant A', 'Variant B']","[0.3, 0.4, 0.3]",Conversion Rate
2,P002,Peak Hour Pass,surge_waiver,1.0,2025-04-26,2025-05-16,All,Cairo,"['Control', 'Variant A', 'Variant B']","[0.3, 0.4, 0.3]",ROI



DRIVERS  |  shape = (5000, 7)


,driver_id,rating,vehicle_type,signup_date,last_active,city,acceptance_rate
0,D00000,3.1,SUV,2025-01-20,2025-01-06 18:23:09.312275,Cairo,0.679555
1,D00001,5.0,Sedan,2023-03-27,2025-04-27 01:44:02.472554,Nairobi,0.548786
2,D00002,4.5,Motorcycle,2024-05-02,2025-03-07 19:24:46.367672,Nairobi,0.593724


### Data types and missing values check

In [9]:
def audit_table(df, name):
    print("\n" + "="*60)
    print(f"AUDIT: {name.upper()}")
    print("-"*60)
    
    # Data types
    print("\nData types:")
    display(df.dtypes.to_frame("dtype"))
    
    # Missing values
    missing = df.isna().sum().sort_values(ascending=False)
    missing = missing[missing > 0]
    print("\nMissing values (only columns with missing):")
    if len(missing) == 0:
        print("No missing values ✅")
    else:
        display(missing.to_frame("missing_count"))

for name, df in datasets.items():
    audit_table(df, name)


AUDIT: RIDERS
------------------------------------------------------------

Data types:


,dtype
user_id,object
signup_date,object
loyalty_status,object
age,float64
city,object
avg_rating_given,float64
churn_prob,float64
referred_by,object



Missing values (only columns with missing):


,missing_count
referred_by,6947



AUDIT: TRIPS
------------------------------------------------------------

Data types:


...


Missing values (only columns with missing):
No missing values ✅

AUDIT: PROMOTIONS
------------------------------------------------------------

Data types:


,dtype
promo_id,object
promo_name,object
promo_type,object
promo_value,float64
start_date,object
end_date,object
target_segment,object
city_scope,object
ab_test_groups,object
test_allocation,object



Missing values (only columns with missing):
No missing values ✅

AUDIT: DRIVERS
------------------------------------------------------------

Data types:


,dtype
driver_id,object
rating,float64
vehicle_type,object
signup_date,object
last_active,object
city,object
acceptance_rate,float64



Missing values (only columns with missing):
No missing values ✅


### Key relationship checks (the join keys)

In [11]:
# Check uniqueness of keys in primary tables
print("Uniqueness checks:")
print("riders.user_id unique?", riders["user_id"].is_unique)
print("drivers.driver_id unique?", drivers["driver_id"].is_unique)

trip_users_not_in_riders = trips.loc[
    ~trips["user_id"].isin(riders["user_id"]),
    "user_id"
].nunique()

session_users_not_in_riders = sessions.loc[
    ~sessions["rider_id"].isin(riders["user_id"]),
    "rider_id"
].nunique()

print("Foreign key sanity checks:")
print("Unique trip users NOT found in riders:", trip_users_not_in_riders)
print("Unique session users NOT found in riders:", session_users_not_in_riders)

Uniqueness checks:
riders.user_id unique? True
drivers.driver_id unique? True
Foreign key sanity checks:
Unique trip users NOT found in riders: 0
Unique session users NOT found in riders: 0


In [12]:
# Standardize naming across datasets
sessions = sessions.rename(columns={"rider_id": "user_id"})

### Identify date columns

In [13]:
# Identify date-like columns in each dataset

date_columns = {
    "riders": ["signup_date"],
    "trips": ["pickup_time", "dropoff_time"],
    "sessions": ["session_time"],
    "promotions": ["start_date", "end_date"],
    "drivers": ["signup_date", "last_active"]
}

date_columns

{'riders': ['signup_date'],
 'trips': ['pickup_time', 'dropoff_time'],
 'sessions': ['session_time'],
 'promotions': ['start_date', 'end_date'],
 'drivers': ['signup_date', 'last_active']}

### Parse dates safely

In [16]:
# Convert date columns to datetime

for col in date_columns["riders"]:
    riders[col] = pd.to_datetime(riders[col], errors="coerce")

for col in date_columns["trips"]:
    trips[col] = pd.to_datetime(trips[col], errors="coerce", utc=True)

for col in date_columns["sessions"]:
    sessions[col] = pd.to_datetime(sessions[col], errors="coerce", utc=True)

for col in date_columns["promotions"]:
    promotions[col] = pd.to_datetime(promotions[col], errors="coerce")

for col in date_columns["drivers"]:
    drivers[col] = pd.to_datetime(drivers[col], errors="coerce")

print("Trips and sessions datetime parsing forced to UTC.")
print("Date parsing completed.")

Trips and sessions datetime parsing forced to UTC.
Date parsing completed.


### Validate date parsing

In [17]:
# Confirm date parsing worked
print("Riders date types:")
display(riders[date_columns["riders"]].dtypes)

print("\nTrips date types:")
display(trips[date_columns["trips"]].dtypes)

print("\nSessions date types:")
display(sessions[date_columns["sessions"]].dtypes)

Riders date types:


signup_date    datetime64[ns]
dtype: object


Trips date types:


pickup_time     datetime64[ns, UTC]
dropoff_time    datetime64[ns, UTC]
dtype: object


Sessions date types:


session_time    datetime64[ns, UTC]
dtype: object

In [18]:
print("Trips date types after UTC fix:")
display(trips[date_columns["trips"]].dtypes)

print("\nSessions date types after UTC fix:")
display(sessions[date_columns["sessions"]].dtypes)

Trips date types after UTC fix:


pickup_time     datetime64[ns, UTC]
dropoff_time    datetime64[ns, UTC]
dtype: object


Sessions date types after UTC fix:


session_time    datetime64[ns, UTC]
dtype: object

In [19]:
trips[date_columns["trips"]].head()

,pickup_time,dropoff_time
0,2024-11-27 16:14:50+00:00,2024-11-27 17:06:50+00:00
1,2024-10-28 22:59:48+00:00,2024-10-28 23:12:48+00:00
2,2025-02-17 03:09:41+00:00,2025-02-17 03:25:41+00:00
3,2024-06-18 17:22:14+00:00,2024-06-18 17:27:14+00:00
4,2024-10-05 07:31:16+00:00,2024-10-05 08:01:16+00:00


### Derive time-based features

In [20]:
# Derive time-based features from trips

trips["trip_date"] = trips["pickup_time"].dt.date
trips["trip_hour"] = trips["pickup_time"].dt.hour
trips["day_of_week"] = trips["pickup_time"].dt.day_name()
trips["is_weekend"] = trips["pickup_time"].dt.weekday >= 5

### Derive session time features

In [21]:
# Derive time-based features from sessions

sessions["session_date"] = sessions["session_time"].dt.date
sessions["session_hour"] = sessions["session_time"].dt.hour
sessions["is_weekend"] = sessions["session_time"].dt.weekday >= 5

### Trips aggregation

In [23]:
# If sessions uses rider_id, standardize it once (makes life easier later)
if "rider_id" in sessions.columns and "user_id" not in sessions.columns:
    sessions = sessions.rename(columns={"rider_id": "user_id"})

# This ensures all tables use user_id, so joins are simple and less error-prone.

### Create trip-level derived fields (for time-of-day analysis + churn)

In [24]:
# Trip-derived time features (non-destructive; original timestamps remain)
trips["trip_date"] = trips["pickup_time"].dt.date
trips["trip_hour"] = trips["pickup_time"].dt.hour
trips["is_weekend"] = trips["pickup_time"].dt.weekday >= 5

In [27]:
print("Trips columns:")
print(sorted(trips.columns.tolist()))

Trips columns:
['city', 'day_of_week', 'driver_id', 'dropoff_lat', 'dropoff_lng', 'dropoff_time', 'fare', 'is_weekend', 'loyalty_status', 'payment_type', 'pickup_lat', 'pickup_lng', 'pickup_time', 'surge_multiplier', 'tip', 'trip_date', 'trip_hour', 'trip_id', 'user_id', 'weather']


### Aggregate trips to 1 row per rider

In [28]:
trips_agg = (
    trips.groupby("user_id")
    .agg(
        total_trips=("trip_id", "count"),
        total_spent=("fare", "sum"),
        avg_fare=("fare", "mean"),
        total_tip=("tip", "sum"),
        avg_tip=("tip", "mean"),
        avg_surge=("surge_multiplier", "mean"),
        last_trip_time=("pickup_time", "max"),
        first_trip_time=("pickup_time", "min"),
        weekend_trip_ratio=("is_weekend", "mean"),
        peak_hour_trip_ratio=("trip_hour", lambda s: s.between(7, 10).mean()),
    )
    .reset_index()
)

trips_agg.head()

,user_id,total_trips,total_spent,avg_fare,total_tip,avg_tip,avg_surge,last_trip_time,first_trip_time,weekend_trip_ratio,peak_hour_trip_ratio
0,R00000,25,366.05,14.642000,4.03,0.161200,1.096000,2025-04-02 14:46:29+00:00,2024-05-01 07:21:52+00:00,0.160000,0.160000
1,R00001,14,180.53,12.895000,0.76,0.054286,1.071429,2025-04-22 04:35:17+00:00,2024-05-10 18:14:41+00:00,0.285714,0.071429
2,R00002,24,378.99,15.791250,5.21,0.217083,1.191667,2025-04-13 00:08:00+00:00,2024-06-18 17:48:24+00:00,0.125000,0.166667
3,R00003,9,121.47,13.496667,0.87,0.096667,1.155556,2025-02-25 04:22:32+00:00,2024-05-15 05:13:12+00:00,0.333333,0.111111
4,R00004,16,268.43,16.776875,9.38,0.586250,1.262500,2025-04-15 05:30:04+00:00,2024-05-23 13:02:45+00:00,0.437500,0.187500


### Validate the aggregation

In [29]:
print("riders rows:", riders.shape[0])
print("trips_agg rows:", trips_agg.shape[0])
print("Unique user_id in trips_agg:", trips_agg["user_id"].nunique())
print("Any duplicate user_id in trips_agg?", trips_agg["user_id"].duplicated().any())

riders rows: 10000
trips_agg rows: 10000
Unique user_id in trips_agg: 10000
Any duplicate user_id in trips_agg? False


In [30]:
trips_agg.shape

(10000, 11)

### Aggregating the Session Table

### Preview sessions and confirm columns

In [31]:
print("sessions shape:", sessions.shape)
print("sessions columns:", sorted(sessions.columns.tolist()))
display(sessions.head(5))

sessions shape: (50000, 11)
sessions columns: ['city', 'converted', 'is_weekend', 'loyalty_status', 'pages_visited', 'session_date', 'session_hour', 'session_id', 'session_time', 'time_on_app', 'user_id']


,session_id,user_id,session_time,time_on_app,pages_visited,converted,city,loyalty_status,session_date,session_hour,is_weekend
0,S000000,R08605,2025-04-27 16:52:06+00:00,79,4,1,Cairo,Bronze,2025-04-27,16,True
1,S000001,R08823,2025-04-27 05:05:22+00:00,101,3,0,Nairobi,Silver,2025-04-27,5,True
2,S000002,R05342,2025-04-27 21:12:25+00:00,12,1,0,Cairo,Bronze,2025-04-27,21,True
3,S000003,R05057,2025-04-27 14:26:25+00:00,19,1,0,Lagos,Silver,2025-04-27,14,True
4,S000004,R09614,2025-04-27 08:17:22+00:00,4,1,0,Lagos,Bronze,2025-04-27,8,True


### Standardize the join key (rider_id renamed to user_id)

In [32]:
# Standardize ID column name to match riders/trips
if "rider_id" in sessions.columns:
    sessions = sessions.rename(columns={"rider_id": "user_id"})

print("Updated sessions columns:", sorted(sessions.columns.tolist()))

Updated sessions columns: ['city', 'converted', 'is_weekend', 'loyalty_status', 'pages_visited', 'session_date', 'session_hour', 'session_id', 'session_time', 'time_on_app', 'user_id']


### Ensure session_time is datetime

In [33]:
# Convert session_time to datetime (handles mixed formats safely)
sessions["session_time"] = pd.to_datetime(sessions["session_time"], errors="coerce", utc=True)

# Optional: remove timezone to make it easier to read (keeps the time)
sessions["session_time"] = sessions["session_time"].dt.tz_convert(None)

print("session_time dtype:", sessions["session_time"].dtype)
print("Invalid session_time rows (NaT):", sessions["session_time"].isna().sum())

session_time dtype: datetime64[ns]
Invalid session_time rows (NaT): 0


### Create time features (for EDA + modeling)

In [34]:
sessions["session_hour"] = sessions["session_time"].dt.hour
sessions["is_weekend"] = sessions["session_time"].dt.weekday >= 5

display(sessions[["user_id", "session_time", "session_hour", "is_weekend"]].head(5))

,user_id,session_time,session_hour,is_weekend
0,R08605,2025-04-27 16:52:06,16,True
1,R08823,2025-04-27 05:05:22,5,True
2,R05342,2025-04-27 21:12:25,21,True
3,R05057,2025-04-27 14:26:25,14,True
4,R09614,2025-04-27 08:17:22,8,True


### Aggregate sessions (1 row per user)

In [35]:
sessions_agg = (
    sessions.groupby("user_id")
    .agg(
        total_sessions=("session_id", "count"),
        total_time_on_app=("time_on_app", "sum"),
        avg_time_on_app=("time_on_app", "mean"),
        total_pages_visited=("pages_visited", "sum"),
        avg_pages_visited=("pages_visited", "mean"),
        conversion_rate=("converted", "mean"),
        last_session_time=("session_time", "max"),
        first_session_time=("session_time", "min"),
        weekend_session_ratio=("is_weekend", "mean"),
        peak_hour_session_ratio=("session_hour", lambda s: s.between(7, 10).mean()),
    )
    .reset_index()
)

display(sessions_agg.head(5))
print("sessions_agg shape:", sessions_agg.shape)

,user_id,total_sessions,total_time_on_app,avg_time_on_app,total_pages_visited,avg_pages_visited,conversion_rate,last_session_time,first_session_time,weekend_session_ratio,peak_hour_session_ratio
0,R00000,4,368,92.000000,12,3.000000,0.25,2025-04-27 16:06:46,2025-04-27 00:49:27,1.0,0.000000
1,R00001,3,524,174.666667,8,2.666667,0.00,2025-04-27 10:33:25,2025-04-27 07:10:07,1.0,1.000000
2,R00002,3,573,191.000000,9,3.000000,0.00,2025-04-27 15:04:35,2025-04-27 07:32:20,1.0,0.666667
3,R00003,3,226,75.333333,5,1.666667,0.00,2025-04-27 09:56:39,2025-04-26 23:28:16,1.0,0.333333
4,R00004,2,34,17.000000,5,2.500000,0.00,2025-04-27 18:28:35,2025-04-27 06:25:27,1.0,0.000000


sessions_agg shape: (9929, 11)


### Validate aggregation

In [36]:
print("riders rows:", riders.shape[0])
print("sessions_agg rows:", sessions_agg.shape[0])
print("Unique user_id in sessions_agg:", sessions_agg["user_id"].nunique())
print("Any duplicate user_id in sessions_agg?", sessions_agg["user_id"].duplicated().any())

riders rows: 10000
sessions_agg rows: 9929
Unique user_id in sessions_agg: 9929
Any duplicate user_id in sessions_agg? False


### Join riders + trips_agg

In [37]:
riders_trips = riders.merge(
    trips_agg,
    on="user_id",
    how="left"
)

print("Rows after joining trips:", riders_trips.shape[0])

Rows after joining trips: 10000


### Join sessions_agg

In [38]:
riders_trips_sessions = riders_trips.merge(
    sessions_agg,
    on="user_id",
    how="left"
)

print("Rows after joining sessions:", riders_trips_sessions.shape[0])

Rows after joining sessions: 10000


### Validate the joined dataset

In [39]:
print("Final dataset shape:", riders_trips_sessions.shape)

print("Any duplicate user_id?",
      riders_trips_sessions["user_id"].duplicated().any())

print("Users with NO sessions:",
      riders_trips_sessions["total_sessions"].isna().sum())

print("Users with NO trips:",
      riders_trips_sessions["total_trips"].isna().sum())

Final dataset shape: (10000, 28)
Any duplicate user_id? False
Users with NO sessions: 71
Users with NO trips: 0


In [40]:
riders_trips_sessions

,user_id,signup_date,loyalty_status,age,city,avg_rating_given,churn_prob,referred_by,total_trips,total_spent,avg_fare,total_tip,avg_tip,avg_surge,last_trip_time,first_trip_time,weekend_trip_ratio,peak_hour_trip_ratio,total_sessions,total_time_on_app,avg_time_on_app,total_pages_visited,avg_pages_visited,conversion_rate,last_session_time,first_session_time,weekend_session_ratio,peak_hour_session_ratio
0,R00000,2025-01-24,Bronze,34.729629,Nairobi,5.0,0.142431,R00001,25,366.05,14.642000,4.03,0.161200,1.096000,2025-04-02 14:46:29+00:00,2024-05-01 07:21:52+00:00,0.160000,0.160000,4.0,368.0,92.000000,12.0,3.000000,0.25,2025-04-27 16:06:46,2025-04-27 00:49:27,1.0,0.000000
1,R00001,2024-09-09,Bronze,34.571020,Nairobi,4.7,0.674161,NaN,14,180.53,12.895000,0.76,0.054286,1.071429,2025-04-22 04:35:17+00:00,2024-05-10 18:14:41+00:00,0.285714,0.071429,3.0,524.0,174.666667,8.0,2.666667,0.00,2025-04-27 10:33:25,2025-04-27 07:10:07,1.0,1.000000
2,R00002,2024-09-07,Bronze,47.133960,Lagos,4.2,0.510379,NaN,24,378.99,15.791250,5.21,0.217083,1.191667,2025-04-13 00:08:00+00:00,2024-06-18 17:48:24+00:00,0.125000,0.166667,3.0,573.0,191.000000,9.0,3.000000,0.00,2025-04-27 15:04:35,2025-04-27 07:32:20,1.0,0.666667
3,R00003,2025-03-17,Bronze,41.658628,Nairobi,4.9,0.244779,NaN,9,121.47,13.496667,0.87,0.096667,1.155556,2025-02-25 04:22:32+00:00,2024-05-15 05:13:12+00:00,0.333333,0.111111,3.0,226.0,75.333333,5.0,1.666667,0.00,2025-04-27 09:56:39,2025-04-26 23:28:16,1.0,0.333333
4,R00004,2024-08-20,Silver,40.681709,Lagos,3.9,0.269960,R00002,16,268.43,16.776875,9.38,0.586250,1.262500,2025-04-15 05:30:04+00:00,2024-05-23 13:02:45+00:00,0.437500,0.187500,2.0,34.0,17.000000,5.0,2.500000,0.00,2025-04-27 18:28:35,2025-04-27 06:25:27,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,R09995,2025-04-26,Silver,20.688176,Lagos,4.4,0.028371,NaN,13,203.15,15.626923,3.22,0.247692,1.146154,2025-04-26 20:07:09+00:00,2024-06-07 05:03:58+00:00,0.384615,0.384615,3.0,175.0,58.333333,11.0,3.666667,0.00,2025-04-27 18:04:09,2025-04-27 06:39:23,1.0,0.333333
9996,R09996,2023-07-26,Silver,18.000000,Cairo,4.1,0.290365,NaN,15,206.71,13.780667,10.64,0.709333,1.033333,2025-04-25 12:34:57+00:00,2024-07-30 15:33:18+00:00,0.133333,0.133333,4.0,159.0,39.750000,11.0,2.750000,0.25,2025-04-27 15:59:54,2025-04-27 08:20:45,1.0,0.500000
9997,R09997,2025-04-18,Bronze,34.197784,Lagos,4.7,0.013147,R02155,18,309.83,17.212778,4.67,0.259444,1.105556,2025-03-12 08:00:59+00:00,2024-04-29 07:30:01+00:00,0.333333,0.388889,5.0,177.0,35.400000,15.0,3.000000,0.20,2025-04-27 16:00:04,2025-04-27 04:09:27,1.0,0.200000
9998,R09998,2024-01-10,Bronze,51.705319,Cairo,3.8,0.253959,R07447,22,342.05,15.547727,10.54,0.479091,1.150000,2025-04-08 14:46:18+00:00,2024-05-07 18:37:03+00:00,0.227273,0.045455,6.0,215.0,35.833333,17.0,2.833333,0.00,2025-04-27 18:28:25,2025-04-27 02:05:47,1.0,0.000000


### Final validation

In [41]:
print("Final dataset shape:", riders_trips_sessions.shape)
print("Unique user_id:", riders_trips_sessions["user_id"].nunique())
print("Any duplicates:", riders_trips_sessions["user_id"].duplicated().any())

Final dataset shape: (10000, 28)
Unique user_id: 10000
Any duplicates: False


### Inspect NaNs

In [42]:
na_summary = (
    riders_trips_sessions
    .isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame("na_ratio")
)

display(na_summary)

,na_ratio
referred_by,0.6947
first_session_time,0.0071
peak_hour_session_ratio,0.0071
weekend_session_ratio,0.0071
avg_time_on_app,0.0071
total_pages_visited,0.0071
avg_pages_visited,0.0071
conversion_rate,0.0071
last_session_time,0.0071
total_sessions,0.0071


In [43]:
### # Create a clean, explicit referral flag
riders_trips_sessions["was_referred"] = riders_trips_sessions["referred_by"].notna().astype(int)

In [44]:
critical_cols = ["user_id", "signup_date", "city", "loyalty_status"]

riders_trips_sessions[critical_cols].isna().sum()

user_id           0
signup_date       0
city              0
loyalty_status    0
dtype: int64

In [45]:
riders_trips_sessions

,user_id,signup_date,loyalty_status,age,city,avg_rating_given,churn_prob,referred_by,total_trips,total_spent,avg_fare,total_tip,avg_tip,avg_surge,last_trip_time,first_trip_time,weekend_trip_ratio,peak_hour_trip_ratio,total_sessions,total_time_on_app,avg_time_on_app,total_pages_visited,avg_pages_visited,conversion_rate,last_session_time,first_session_time,weekend_session_ratio,peak_hour_session_ratio,was_referred
0,R00000,2025-01-24,Bronze,34.729629,Nairobi,5.0,0.142431,R00001,25,366.05,14.642000,4.03,0.161200,1.096000,2025-04-02 14:46:29+00:00,2024-05-01 07:21:52+00:00,0.160000,0.160000,4.0,368.0,92.000000,12.0,3.000000,0.25,2025-04-27 16:06:46,2025-04-27 00:49:27,1.0,0.000000,1
1,R00001,2024-09-09,Bronze,34.571020,Nairobi,4.7,0.674161,NaN,14,180.53,12.895000,0.76,0.054286,1.071429,2025-04-22 04:35:17+00:00,2024-05-10 18:14:41+00:00,0.285714,0.071429,3.0,524.0,174.666667,8.0,2.666667,0.00,2025-04-27 10:33:25,2025-04-27 07:10:07,1.0,1.000000,0
2,R00002,2024-09-07,Bronze,47.133960,Lagos,4.2,0.510379,NaN,24,378.99,15.791250,5.21,0.217083,1.191667,2025-04-13 00:08:00+00:00,2024-06-18 17:48:24+00:00,0.125000,0.166667,3.0,573.0,191.000000,9.0,3.000000,0.00,2025-04-27 15:04:35,2025-04-27 07:32:20,1.0,0.666667,0
3,R00003,2025-03-17,Bronze,41.658628,Nairobi,4.9,0.244779,NaN,9,121.47,13.496667,0.87,0.096667,1.155556,2025-02-25 04:22:32+00:00,2024-05-15 05:13:12+00:00,0.333333,0.111111,3.0,226.0,75.333333,5.0,1.666667,0.00,2025-04-27 09:56:39,2025-04-26 23:28:16,1.0,0.333333,0
4,R00004,2024-08-20,Silver,40.681709,Lagos,3.9,0.269960,R00002,16,268.43,16.776875,9.38,0.586250,1.262500,2025-04-15 05:30:04+00:00,2024-05-23 13:02:45+00:00,0.437500,0.187500,2.0,34.0,17.000000,5.0,2.500000,0.00,2025-04-27 18:28:35,2025-04-27 06:25:27,1.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,R09995,2025-04-26,Silver,20.688176,Lagos,4.4,0.028371,NaN,13,203.15,15.626923,3.22,0.247692,1.146154,2025-04-26 20:07:09+00:00,2024-06-07 05:03:58+00:00,0.384615,0.384615,3.0,175.0,58.333333,11.0,3.666667,0.00,2025-04-27 18:04:09,2025-04-27 06:39:23,1.0,0.333333,0
9996,R09996,2023-07-26,Silver,18.000000,Cairo,4.1,0.290365,NaN,15,206.71,13.780667,10.64,0.709333,1.033333,2025-04-25 12:34:57+00:00,2024-07-30 15:33:18+00:00,0.133333,0.133333,4.0,159.0,39.750000,11.0,2.750000,0.25,2025-04-27 15:59:54,2025-04-27 08:20:45,1.0,0.500000,0
9997,R09997,2025-04-18,Bronze,34.197784,Lagos,4.7,0.013147,R02155,18,309.83,17.212778,4.67,0.259444,1.105556,2025-03-12 08:00:59+00:00,2024-04-29 07:30:01+00:00,0.333333,0.388889,5.0,177.0,35.400000,15.0,3.000000,0.20,2025-04-27 16:00:04,2025-04-27 04:09:27,1.0,0.200000,1
9998,R09998,2024-01-10,Bronze,51.705319,Cairo,3.8,0.253959,R07447,22,342.05,15.547727,10.54,0.479091,1.150000,2025-04-08 14:46:18+00:00,2024-05-07 18:37:03+00:00,0.227273,0.045455,6.0,215.0,35.833333,17.0,2.833333,0.00,2025-04-27 18:28:25,2025-04-27 02:05:47,1.0,0.000000,1


### Saving the dataset

In [46]:
OUTPUT_PATH = "../data/riders_trips_session.csv"

riders_trips_sessions.to_csv(OUTPUT_PATH, index=False)

print("Saved feature-assembled dataset with intentional NaNs preserved.")

Saved feature-assembled dataset with intentional NaNs preserved.
